# example notebook
Databricks notebook fixture for doxcer tests (bronze to gold flow).

In [ ]:
dbutils.widgets.text("p_load_date", "2026-02-13")
dbutils.widgets.dropdown("p_force_full_load", "false", ["true", "false"])

p_load_date = dbutils.widgets.get("p_load_date")
p_force_full_load = dbutils.widgets.get("p_force_full_load").lower() == "true"
print(f"Running for load date: {p_load_date}, full load: {p_force_full_load}")

In [ ]:
from pyspark.sql import functions as F

bronze_orders_df = spark.createDataFrame(
    [
        ("1001", "C001", "2026-02-13", 125.50, "completed"),
        ("1002", "C002", "2026-02-13", 20.00, "cancelled"),
        ("1003", "C001", "2026-02-13", 88.00, "completed")
    ],
    ["order_id", "customer_id", "order_date", "order_amount", "status"]
)

In [ ]:
gold_fct_daily_customer_sales_df = (
    bronze_orders_df
    .filter(F.col("status") == "completed")
    .groupBy("order_date", "customer_id")
    .agg(
        F.sum("order_amount").alias("daily_revenue"),
        F.count("*").alias("order_count")
    )
)

display(gold_fct_daily_customer_sales_df)

In [ ]:
(
    gold_fct_daily_customer_sales_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("gold.fct_daily_customer_sales")
)

dbutils.notebook.exit(f"success|load_date={p_load_date}")